In [ ]:
# | default_exp _experiments.loan

# Loan


Lending club loan *data*
contains complete loan data for all loans
issued through 2007-2015 of several banks. Each data point is a 28-dimensional feature including
the current loan status, latest payment information, and other additional features. The task is to
predict loan defaulters given the feature vector. The possibility of loan default should be nondecreasing w.r.t. number of public record bankruptcies, Debt-to-Income ratio, and
non-increasing w.r.t. credit score, length of employment, annual income. Thus the `monotonicity_indicator` corrsponding to these features are set to 1.


References:

1. https://www.kaggle.com/wendykan/lending-club-loan-data (Note: Currently, the dataset seems to be withdrawn from kaggle)

In [ ]:
monotonicity_indicator = {
    f"feature_{i}": mi for i, mi in enumerate([-1, 1, -1, -1, 1] + [0] * 23)
}

## Running in Google Colab

<a href="https://colab.research.google.com/github/airtai/monotonic-nn/blob/main/nbs/experiments/Loan.ipynb" target=”_blank”>
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" />
</a>


In [ ]:
# | hide

from IPython.display import Markdown, display_markdown

try:
    import google.colab

    in_colab = True
except:
    in_colab = False

if in_colab:
    display(
        Markdown(
            """
### If you see this message, you are running in Google Colab
Along with this interactive tutorial the content of this notebook is organized and formatted for documentation purpuoses. 

You can ignore the '# | hide', '# | notest' and '# | echo: false' comments, they are not important for the tutorial.
    """
        )
    )

In [ ]:
# | hide

if in_colab:
    !pip install "monotonic-nn[experiments]"

In [ ]:
# | include: false

from airt.keras.experiments import (
    create_tuner_stats,
    find_hyperparameters,
    get_train_n_test_data,
)

In [ ]:
# | include: false
import shutil
from os import environ

import tensorflow as tf

In [ ]:
# | include: false

environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

gpus = tf.config.list_physical_devices("GPU")
if gpus:
    # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.set_visible_devices(gpus[2], "GPU")
        logical_gpus = tf.config.list_logical_devices("GPU")
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)

3 Physical GPUs, 1 Logical GPU


These are a few examples of the dataset:

In [ ]:
# | echo: false

train_df, test_df = get_train_n_test_data(dataset_name="loan")
display(train_df.head().T.style)

,0,1,2,3,4
feature_0,0.833333,1.000000,0.666667,0.333333,0.666667
feature_1,0.000000,0.000000,0.000000,0.000000,0.000000
feature_2,0.400000,1.000000,0.800000,0.500000,0.700000
feature_3,0.005263,0.003474,0.005263,0.007158,0.006842
feature_4,0.005185,0.023804,0.029700,0.024434,0.021962
feature_5,0.185751,0.134860,0.236641,0.745547,0.440204
feature_6,0.240654,0.036215,0.271807,0.778037,0.260125
feature_7,0.000000,0.000000,0.000000,1.000000,0.000000
feature_8,0.000000,0.000000,0.000000,0.000000,0.000000
feature_9,0.000000,0.000000,1.000000,0.000000,1.000000


## Hyperparameter search

The choice of the batch size and the maximum number of epochs depends on the dataset size. For this dataset, we use the following values:

In [ ]:
batch_size = 256
max_epochs = 20

We use the Type-2 architecture built using `MonoDense` layer with the following set of hyperparameters ranges:

In [ ]:
def hp_params_f(hp):
    return dict(
        units=hp.Int("units", min_value=4, max_value=32, step=1),
        n_layers=hp.Int("n_layers", min_value=1, max_value=2),
        activation=hp.Choice("activation", values=["elu"]),
        learning_rate=hp.Float(
            "learning_rate", min_value=1e-4, max_value=1e-2, sampling="log"
        ),
        weight_decay=hp.Float(
            "weight_decay", min_value=3e-2, max_value=0.3, sampling="log"
        ),
        dropout=hp.Float("dropout", min_value=0.0, max_value=0.5, sampling="linear"),
        decay_rate=hp.Float(
            "decay_rate", min_value=0.8, max_value=1.0, sampling="reverse_log"
        ),
    )

The following fixed parameters are used to build the Type-2 architecture for this dataset:

- `final_activation` is used to build the final layer for regression problem (set to `None`) or for the classification problem (`"sigmoid"`),

- `loss` is used for training regression (`"mse"`) or classification (`"binary_crossentropy"`) problem, and

- `metrics` denotes metrics used to compare with previosly published results: `"accuracy"` for classification and "`mse`" or "`rmse`" for regression.

Parameters `objective` and `direction` are used by the tuner such that `objective=f"val_{metrics}"` and direction is either `"min` or `"max"`.

Parameters `max_trials` denotes the number of trial performed buy the tuner, `patience` is the number of epochs allowed to perform worst than the best one before stopping the current trial. The parameter `execution_per_trial` denotes the number of runs before calculating the results of a trial, it should be set to value greater than 1 for small datasets that have high variance in results.

In [ ]:
final_activation = None
loss = "binary_crossentropy"
metrics = "accuracy"
objective = "val_accuracy"
direction = "max"
max_trials = 50
executions_per_trial = 1
patience = 5

In [ ]:
# | include: false

# uncomment and wait for a long time to find hyperparameters
find_hyperparams = False

if find_hyperparams:
    tuner = find_hyperparameters(
        "loan",
        dir_root="tuner-2",
        monotonicity_indicator=monotonicity_indicator,
        hp_params_f=hp_params_f,
        final_activation=final_activation,
        loss=loss,
        metrics=metrics,
        objective=objective,
        direction=direction,
        max_trials=max_trials,
        patience=patience,
        executions_per_trial=executions_per_trial,
        batch_size=batch_size,
        max_epochs=max_epochs,
    )
else:
    tuner = None

In [ ]:
# | include: false

if tuner is not None:
    stats = create_tuner_stats(
        tuner,
        batch_size=batch_size,
        max_epochs=max_epochs,
    )

The following table describes the best models and their hyperparameters found by the tuner:

In [ ]:
# | echo: false

if tuner is not None:
    df = stats.sort_values(
        by=f"{objective}_mean", ascending=(direction == "min")
    ).head()

    display(df.reset_index(drop=True).T.style)

In [ ]:
# | include: false
if tuner is not None:
    print(df.to_latex(index=False))

## The optimal model

These are the best hyperparameters found by previous runs of the tuner:

In [ ]:
def final_hp_params_f(hp):
    return dict(
        units=hp.Fixed("units", value=8),
        n_layers=hp.Fixed("n_layers", 2),
        activation=hp.Fixed("activation", value="elu"),
        learning_rate=hp.Fixed("learning_rate", value=0.008),
        weight_decay=hp.Fixed("weight_decay", value=0.0),
        dropout=hp.Fixed("dropout", value=0.0),
        decay_rate=hp.Fixed("decay_rate", value=1.0),
    )

In [ ]:
# | include: false
# | notest


shutil.rmtree("tuner_final/loan", ignore_errors=True)

final_tuner = find_hyperparameters(
    "loan",
    monotonicity_indicator=monotonicity_indicator,
    hp_params_f=final_hp_params_f,
    max_trials=1,
    final_activation=final_activation,
    loss=loss,
    metrics=metrics,
    objective=objective,
    direction=direction,
    batch_size=batch_size,
    max_epochs=max_epochs,
    patience=patience,
    executions_per_trial=1,
    dir_root="tuner_final",
)

Trial 1 Complete [00h 03m 52s]
val_accuracy: 0.6518259048461914

Best val_accuracy So Far: 0.6518259048461914
Total elapsed time: 00h 03m 52s
INFO:tensorflow:Oracle triggered exit


In [ ]:
# | include: false
# | notest

final_stats = create_tuner_stats(
    final_tuner,
    batch_size=batch_size,
    max_epochs=max_epochs,
)

,units,n_layers,activation,learning_rate,weight_decay,dropout,decay_rate,val_accuracy_mean,val_accuracy_std,val_accuracy_min,val_accuracy_max,params
0,8,2,elu,0.008,0.0,0.0,1.0,0.652917,0.000085,0.652851,0.653065,577


The final evaluation of the optimal model:

In [ ]:
# | echo: false
# | notest

final_stats.T.style

,0
units,8
n_layers,2
activation,elu
learning_rate,0.008000
weight_decay,0.000000
dropout,0.000000
decay_rate,1.000000
val_accuracy_mean,0.652917
val_accuracy_std,0.000085
val_accuracy_min,0.652851
